In [1]:
import os
import subprocess
import sys
import shutil

In [2]:
def convertSolution(run_name, year, csv_type):      
    # Calling sltoht.exe to convert .sl4 to .csv (See GEMPACK mannual 39.4 for details)
    # Format: sltoht.exe -[CSV type] -map=[map file name].map [sl4 file name] [output file name].csv
    subprocess.run(['sltoht.exe',\
                    '-' + csv_type,\
                    '-map=' + run_name + year + '.map',\
                    run_name + year,\
                    run_name + year + '.csv'])

In [3]:
# Creates .map file via given variables
# Note: variable of the form [var1:var2:var3:var4] will put var1-4 in a table as columns, while [var1, var2, var3, ...] will put each one in its own table.
#       both format can be used together, [var1:var2:var3:var4] will simply be treated as a single item.
# Example: [var1:var2:var3, var4, var5:var6] will put var1-3 side-by-side on the 1st table, var4 by itself on the 2nd, and var5-6 side-by-side on the 3rd
def makeMap(run_name, year, variables=[]):
    if(os.path.isfile(run_name + year + '.map') and len(variables) > 0):
        os.remove(run_name + year + '.map')
    try:
        with open(run_name + year + '.map', 'w+') as mf:
            for var in variables:
                mf.write(str(var) + '\n')
    except:
        print('Error writing .map file.')
        raise

In [4]:
# Cleans the SES format .csv file to keep it pandas-friendly
# Specifically, removes comment lines and trailing space/commas
def cleanSES(run_name, year):
    try:
        shutil.copy(run_name + year + '.csv', run_name + year + '-copy.csv')
        with open(run_name + year + '.csv', 'w+') as tf:
            with open(run_name + year + '-copy.csv', 'r') as sf:
                lines = sf.readlines()
                for line in lines:
                    if(len(line.lstrip()) > 0 and line.lstrip()[0] == '!'): # Skipping comment lines
                        continue
                    tf.write(line.rstrip(', \n') + '\n')                    # Copying while stripping trailing commas and spaces
        os.remove(run_name + year + '-copy.csv')
    except:
        print('Error cleaning .csv file.')
        raise

In [5]:
# Different sltoht.exe options produce CSV with different format 
# which requires different cleaning procedures
def cleanCSV(run_name , year, csv_type):
    csv_type = csv_type.upper()
    if(csv_type == 'SES'):
        cleanSES(run_name, year)
    elif(csv_type == 'SSS'):
        cleanSSS(run_name, year)
    else:
        print('Unknown CSV type')

In [6]:
run_name = 'SolutionFile/basb-brrr-'
year = '2024'
variables=['vgdp:EV']
csv_type='SES'

In [7]:
makeMap(run_name, year, variables)
convertSolution(run_name, year, csv_type)
cleanCSV(run_name, year, csv_type)